# Assignment 2 ([M4LP](https://osiris.uu.nl/osiris_student_uuprd/OnderwijsCatalogusSelect.do?selectie=cursus&collegejaar=2022&cursus=KI3V21001))

The assignment covers lexical semantics and reasoning.  
<font color="red">**The rules to follow**:</font>  
* Don't delete any initially provided cells, either text or code cells (but you should delete certain lines in the cells, continue reading).
* Don't delete the exercise code header `#...# EXERCISE n #..#` lines and the `# TEST` lines. 
* Don't change the names of provided functions and variables. 
* If you skip an exercise, then delete all lines in the cell following the header `#...# EXERCISE n #..#` but leave its backup part--the line with `IFSKIPPED` and its following lines.
* If you solve an exercise, then delete its corresponding backup part starting with `IFSKIPPED` and the following lines. 
* Use global vars throughout your code and change only those globals vars that are explicitly instructed. 
* For `#TEST` cells, if its output is coming from your code, then leave it; otherwise clear the output of the cell as it is uninformative and clutters the ipynb. 
* For Text cells, you are expected to insert your input only in the cells that come with a red section title. 
* Name the ipynb file with your group number, e.g., `01.ipynb` or `11.ipynb`.

<font color="red">You following these rules helps us to grade the submissions relatively efficiently. If these rules are violated, a submission will be subject to penalty points.</font>  

<font color="red">**IMPORTANT**</font>: you are strongly encouraged to use Google Colab when solving the exercises. Setting the common environment prevents students and teachers from various headaches related to cross-platform variations, module/package versioning, and unpredicted behaviour of the code. In this way, we try that you spend as much time as possible on coding during the course rather than on installations. Moreover, colab notebooks are very practical for group collaboration as they come with version history and several persons can work on the same notebook (not simultaneously though).  
You are still free to solve the exercises on your own machine but in the end, make sure that your solutions also work in the colab environment. 

by L.abzianidze@uu.nl

# <font color="red">Contributions</font>

~~Delete this text and write instead of it your:~~
* ~~group number (same as the file name, for sanity chack)~~
* ~~a list of group members names (NOT student IDs)~~
* ~~who contributed to which exercises (you don't need to be very detailed)~~ 

# Installation

In [ ]:
import spacy
if spacy.__version__ != '3.5.2':
    print(f"spaCy v={spacy.__version__} but it should be 3.5.2\nForce install 3.5.2 with the next cell")

In [ ]:
# may require environment restart
# !pip install spacy==3.5.2

In [ ]:
# may require environment restart
!python -m spacy download en_core_web_md

In [ ]:
# assigntools package is a course specific collection of useful tools 
!rm -fr assigntools # helps to rerun this cell witthout errors, if recloning needed 
! git clone https://github.com/kovvalsky/assigntools.git

In [ ]:
import nltk
import sys
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
import spacy
# Course-specific package
from assigntools.M4LP.A1 import read_pickle, write_pickle
from assigntools.M4LP.A2 import evaluate_contextual_lex_rel, taged2offsets, show_tableau, LangPro

In [ ]:
# TEST
print(f"spaCy version: {spacy.__version__}")
print(f"Python version: {sys.version}")
print(f"NLTK version: {nltk.__version__}")

spaCy version: 3.5.2
Python version: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
NLTK version: 3.8.1


# User modules

Import all modules here what you might need in addition to what is already imported.

In [ ]:
# IMPORT ALL ADDED AND NECESSARY MODULES HERE (IF ANY)




# WordNet

The code shows how to get the noun(!) sysnsets for `lecture` and for each sysnset to show its definition and examples and to list its word senses. This code attempts to show how to get all the info what is displayed in the [online results](http://wordnetweb.princeton.edu/perl/webwn?s=lecture&sub=Search+WordNet&o2=&o0=1&o8=1&o1=1&o7=1&o5=&o9=&o6=&o3=&o4=&h=00000).
More details about `wn.Synset` and `wn.Lemma` classes can be found [here](https://www.nltk.org/_modules/nltk/corpus/reader/wordnet.html) and [here](https://www.nltk.org/howto/wordnet.html).  
<font color="red">It is important that you understand this code well as it will help you with other WordNet exercises</font>.  
Note that NLTK uses WordNet 3.0 while the online browser 3.1. So, if there are some mismatches between these two, one of the reasons can be different versions.  
If you prefer more graphical visualization of wordnet, check [visuwords](https://visuwords.com/). Warning: it might get pretty messy when large synsets are explored.  
Definitions of certain technical terms of WordNet can be looked up [here](https://wordnet.princeton.edu/documentation/wngloss7wn).

In [ ]:
for synset in wn.synsets('lecture', pos=wn.NOUN):
    all_examples = ','.join([ f'"{e}"' for e in synset.examples() ])
    print(f'{synset}:\n\t({synset.definition()})\n\t{all_examples}')
    for l in synset.lemmas():
        # find a sense number of the lemma of the target synset.
        # For this, first, we find all synsets for this lemma and 
        # in this synset list we find the position+1 of the target synset 
        all_synsets_of_l = wn.synsets(l.name())
        sense_num = all_synsets_of_l.index(l.synset()) + 1
        print(f"\t{l.name()}#{sense_num}")

Synset('lecture.n.01'):
	(a speech that is open to the public)
	"he attended a lecture on telecommunications"
	lecture#1
	public_lecture#1
	talk#4
Synset('lecture.n.02'):
	(a lengthy rebuke)
	"a good lecture was my father's idea of discipline","the teacher gave him a talking to"
	lecture#2
	speech#6
	talking_to#1
Synset('lecture.n.03'):
	(teaching by giving a discourse on some subject (typically to a class))
	
	lecture#3
	lecturing#1


# Contextual lexical relation

We will be working with the `context_ppdb` dataset from [Shwartz et al (2016)](https://aclanthology.org/S16-2013/). The examples from the dataset can be found in Table 1 of the paper and in the dataset itself. It is also recommended to read `README.txt` found in the dataset archive. We will use `dataset.tsv` file for the exercise (as we are not doing any model training).

In a nutshell, the task is to guess a lexical relation that holds between the senses of `x` and `y` that they have in their corresponding contexts. We will divide the task into two tasks. First will be the word sense disambiguation (WSD) for `x/y` in their contexts, and the second will be to predict a lexical relation between the word senses based on WordNet.

In [ ]:
# Downloading the context_ppdb dataset
!wget https://naturallogic.pro/_files_/download/context_ppdb_fine_human_precise.zip
!unzip -o context_ppdb_fine_human_precise.zip -d context_ppdb

In [ ]:
# viewing top 10 lines of the dataset
!head context_ppdb/dataset.tsv

bicycle	riding	Bolotta recounted finding 22 sharpened <x>bicycle</x> spokes jabbed into the lawn while she was out with the lawn mower.	A lesser known work of Hopper's, 'Bridal Path' shows a horseback <y>riding</y> path in Central Park.	other-related	0.6
photo	picture	Some of the mission will include examining the surface for sources of water, and taking comparison <x>photos</x> of the light side and dark sides of the surface.	The trial judge, His Honour Judge Michael Murphy QC, who had previously ordered the jury not to consult the Internet, did not halt the prosecution as he felt 'satisfied' the jury hadn't seen the <y>picture</y>.	equivalence	1.0
catch	fish	Department officer Chris Mitchell says the fishermen had traversed 200 miles of Commonwealth waters and were inside the three nautical mile state limit when they were <x>caught</x>.	A 375 million-year old fossilised umbilical cord indicates that placoderm, thought to be ancestors of modern <y>fish</y>, are actually closer to shar

## Ex1[?pt]: Reading data

Write `read_data` function to read data from the file. Before reading the data from the file, understand the content & format of the file. 
It is not a good practice when the original texts are changed, like in this dataset, where tags `<x>` and `<y>` are used to mark occurences of target words. Because of this, we first need to clean the texts from the tags but save the character offsets of the tagged tokens (to avoid information loss). The cleaned sentences will later be used as an input to spaCy.  
To help you with replacing tagged word info with character offsets, we provide you a ready function [tagged2offsets](https://github.com/kovvalsky/assigntools/blob/main/M4LP/A2.py) that does this (it was imported in the beginning). 

In [ ]:
#TEST: a real example from the dataset
s = "Studies have shown that <x>drinks</x>, especially sweetened, contribute to obesity among adults and children, leading to diseases like diabetes. 'Eight out of ten <x>drinks</x> sold in California public schools are sports <x>drinks</x>,' Padilla said, citing information from the California Department of Public Health."
cleaned_s, offsets = taged2offsets("x", s)
print(cleaned_s)
for start, end in offsets:
    print(f"{cleaned_s[start:end]} at {start}:{end}")



```
Studies have shown that drinks, especially sweetened, contribute to obesity among adults and children, leading to diseases like diabetes. 'Eight out of ten drinks sold in California public schools are sports drinks,' Padilla said, citing information from the California Department of Public Health.
drinks at 24:30
drinks at 156:162
drinks at 208:214
```



In [ ]:
################################################################################
################################## EXERCISE 1 ##################################
################################################################################

def read_data(file_path):
    """ Read the samples from the data file.
        Return a list of samples, where each list element is a dictionary
        {'x':(x, tag_free_context_x, offsets_x), 'y':(y, tag_free_context_y, offsets_y), 
         'r':semantic_relation, 'c':confidence }.
        x and y are target words for left and right contexts while 
        tag_free_context_* are corresponding context sentences without <x/y> tags.
        offsets_x/y give positions of tagged words in the tag_free_context_x/y  
    """
    # use provided taged2offsets function to get offsets of tagged words


In [ ]:
data = read_data('context_ppdb/dataset.tsv')

#IFSKIPPED
# data = read_pickle('data.pkl') 

In [ ]:
#TEST EX1: compare(!) this output to the corresponding lines in the data file
# in order to better understand what kind of (weird) samples the data contains
print(data[0])
print(data[667])
print(data[766])
print(data[3338])
print(f"The data size = {len(data)}")



```
{'x': ('bicycle', 'Bolotta recounted finding 22 sharpened bicycle spokes jabbed into the lawn while she was out with the lawn mower.', [(39, 46)]), 'y': ('riding', "A lesser known work of Hopper's, 'Bridal Path' shows a horseback riding path in Central Park.", [(65, 71)]), 'r': 'other-related', 'c': '0.6'}
{'x': ('girl', 'One girl is still missing.', [(4, 8)]), 'y': ('woman', "Activists have sought women's right to vote in Saudi Arabia for years.", [(22, 27)]), 'r': 'other-related', 'c': '0.6'}
{'x': ('family', 'Father Yus Mawengkang, a Catholic priest in Papua, said the family had been transported to the island on fishing boats and hoped that they would be collected by Australian authorities.', [(60, 66)]), 'y': ('child', 'He is survived by three children, along with grandchildren and great-grandchildren.', [(24, 32), (50, 58), (74, 82)]), 'r': 'other-related', 'c': '0.8'}
{'x': ('goal', 'Scientific goals.', [(11, 16)]), 'y': ('soccer', "Fans of the Netherlands' soccer team were wildly celebrating too.", [(25, 31)]), 'r': 'independent', 'c': '0.8'}
The data size = 3404
```



In [ ]:
#TEST: we also provide you with a function that evaluates predicted relations
# wrt the gold relations in data. Let's see if one always predicts 'independent'
# relation, what will be its accuracy and confusion matrix wrt the gold relations
evaluate_contextual_lex_rel(['independent']*len(data), data, draw=True)
# Note that 'independent' is the majority class baseline 

## Ex2[?pt]: Processing with spaCy

In order to guess the sense of a word in a context, it is helpful to know its POS tag. To find out the POS tags, we process context sentences with spaCy. To make processing faster, you can use `.pipe()` method (see Assignmnet 1) to parse all sentences in a single run.  
Note that we use `en_core_web_md` as it is the smallest model that comes with word vectors (and we will use them in latter exercises). 

In [ ]:
################################################################################
################################## EXERCISE 2 ##################################
################################################################################

def spacy_process(data):
    """ Takes the data variable as an input and processes the sentences with spacy.
        Returns a list of samples, where each sample is a dictionary
        {'x': (pos_x, doc_x), 'y': (pos_y, doc_y)}, where pos and doc are
        spaCy's pos tag (token.pos_) for a corresponging target word and 
        Doc object of the corresponding sentecne. 
        Sometimes target words have several occurences in a sentence 
        that might get different pos tags. In this case, for the sake of determinism,
        pick the first from the alphabetical order, e.g., ADJ is prefered over NOUN.
        The list order follows the order of samples in the data  
    """
    nlp = spacy.load("en_core_web_md")



In [ ]:
#TEST
data[0]



```
{'x': ('bicycle',
  'Bolotta recounted finding 22 sharpened bicycle spokes jabbed into the lawn while she was out with the lawn mower.',
  [(39, 46)]),
 'y': ('riding',
  "A lesser known work of Hopper's, 'Bridal Path' shows a horseback riding path in Central Park.",
  [(65, 71)]),
 'r': 'other-related',
 'c': '0.6'}
```



In [ ]:
# takes ~40 sec with .pipe()
docs_data = spacy_process(data)

#IFSKIPPED
# docs_data = read_pickle('docs_data.pkl')

In [ ]:
#TEST
print(data[766]['y'])
print(docs_data[766]['y'])
print(f"{'-':-^20}")
print(data[3068]['x'])
print(docs_data[3068]['x'])
print(f"{'-':-^20}")
print(data[3338]['x'])
print(docs_data[3338]['x'])



```
('child', 'He is survived by three children, along with grandchildren and great-grandchildren.', [(24, 32), (50, 58), (74, 82)])
('NOUN', He is survived by three children, along with grandchildren and great-grandchildren.)
--------------------
('front', "Several officers in white forensic suits were examining the barricaded area and a huge white tent was erected in front of the house's front door.", [(113, 118), (134, 139)])
('ADJ', Several officers in white forensic suits were examining the barricaded area and a huge white tent was erected in front of the house's front door.)
--------------------
('goal', 'Scientific goals.', [(11, 16)])
('NOUN', Scientific goals.)
```



In [ ]:
#TEST EX2 
#checking what is the counts of pos tags assigned by spaCy en_core_web_md(!) to the target words 
word_pos_cnt = Counter(s[i][0] for s in docs_data for i in 'xy' )
print(word_pos_cnt)
print(f"{sum(word_pos_cnt.values())} pos tagged words show that we have tags for all target words")



```
Counter({'NOUN': 4921, 'VERB': 946, 'PROPN': 765, 'ADJ': 174, 'INTJ': 2})
6808 pos tagged words show that we have tags for all target words
```



## WSD

After we have processed context sentences, we have POS tags for the target words. This will help us to narrow down the search space of word senses. Now it is time to do Word Sense Disambiguation (WSD) for each word in its context sentence. You are supposed to write three functions that predict WordNet sense for each word in the context for the entire data. Each of these functions take `data` and `docs_data` and returns a list of dictionaries with synsets of `x` and `y` words. The functions will differ the way they predict senses. 


* `most_frequent_sense()` - picks the first sense of the word, which is also the most frequent sense of the word.pos (i.e., word.pos.01).
* `simple_lesk_sense()` - picks the sense whose description and/or examples has largest overlap with the context sentence.
* `vector_lesk_sesne()` - picks the sense whose description and/or examples vector is most similar to the vector of the context sentence.

<font color="red">IMPORTANT</font>: WordNet covers the words belonging to four classes (noun, verb, adjective, and adverb). In case spaCy assigned a POS tag that is none of these four, the wsd functions should be able to have some fallback pos tag in such cases to avoid runtime errors. Noun is a good option for such a fallback pos tag. For example, if a word gets proper name POS tag, a wsd can assume that its tag is Noun.

Note that whether the WordNet description and examples are used alone or together, it is up to you. If you want to have a relativelyt high-performing system in the end, you might want to explore different settings.  

Potentially useful links: [link1](https://github.com/Akirato/Lesk-Algorithm/blob/master/leskAlgorithm.py), [link2](https://medium.com/analytics-vidhya/comparative-word-sense-disambiguation-1c3f0f4be1fa), [link3](https://www.nltk.org/howto/wsd.html)



## Ex3[?pt]: Most frequent sense

In [ ]:
################################################################################
################################## EXERCISE 3 ##################################
################################################################################

def most_frequent_sense(data, docs_data):
    """ Takes data and docs_data, and return a list of dictionaries
        {'x': synset_of_x, 'y': synset_of_y } where synset_of_x/y is 
        the first sense of x/y in the context_x/y.
        Note that pos tag of x/y is retrieved from docs_data and used
        to restric the possible senses for a word.
        If synset_of_x/y cannot be found (e.g., x/y is an unknown word), 
        return None for the corresponding x/y key.
        The order in the returned list follows the orders in data and docs_data.
    """


In [ ]:
MFS_data = most_frequent_sense(data, docs_data)

In [ ]:
#TEST EX3: note that in this output, "sit" in 628_x is identifiet as Noun but
# WordNet doesn't have noun sense of "sit", hence the synset is None
for i in [628, 766, 3338]:
    for xy in 'xy':
        print(data[i][xy][:2])
        print(docs_data[i][xy])
        print(MFS_data[i][xy])
    print(f"{'':-^20}")



```
('sit', 'In West Bengal, the ruling Left Front organised sit-ins before the offices of oil PSUs.')
('NOUN', In West Bengal, the ruling Left Front organised sit-ins before the offices of oil PSUs.)
None
('stand', 'It was worth my while to try to stand up and punch with him.')
('VERB', It was worth my while to try to stand up and punch with him.)
Synset('stand.v.01')
--------------------
('family', 'Father Yus Mawengkang, a Catholic priest in Papua, said the family had been transported to the island on fishing boats and hoped that they would be collected by Australian authorities.')
('NOUN', Father Yus Mawengkang, a Catholic priest in Papua, said the family had been transported to the island on fishing boats and hoped that they would be collected by Australian authorities.)
Synset('family.n.01')
('child', 'He is survived by three children, along with grandchildren and great-grandchildren.')
('NOUN', He is survived by three children, along with grandchildren and great-grandchildren.)
Synset('child.n.01')
--------------------
('goal', 'Scientific goals.')
('NOUN', Scientific goals.)
Synset('goal.n.01')
('soccer', "Fans of the Netherlands' soccer team were wildly celebrating too.")
('NOUN', Fans of the Netherlands' soccer team were wildly celebrating too.)
Synset('soccer.n.01')
--------------------
```



## Ex4[?pt]: Simple Lesk

Simplest is to use [NLTK's Lesk](https://www.nltk.org/howto/wsd.html). You can read more about Lesk Algorithm [here](https://en.wikipedia.org/wiki/Lesk_algorithm). You are also welcome to write your own Lesk version as NLTK's one is not performing well.

In [ ]:
################################################################################
################################## EXERCISE 4 ##################################
################################################################################

def simple_lesk_sense(data, docs_data):
    """ Takes data and docs_data, and return a list of dictionaries
        {'x': synset_of_x, 'y': synset_of_y } where synset_of_x/y is 
        the sense of x/y in the context_x/y detecting by Lesk-like algorithm,
        which selects a sense based on the size of the overlap between the
        context and the definition and/or examples of the possible synsets.
        Note that pos tag of x/y is retrieved from docs_data and used
        to restric the possible senses for a word.
        The order in the returned list follows the orders in data and docs_data.
    """


In [ ]:
SIMPLE_LESK_data = simple_lesk_sense(data, docs_data)

In [ ]:
#TEST EX4
# These are the same data samples as the ones above.
# compare sense predictions from this to the previous function(s) and find the difference
# IMPORTANT: depending on certain choice points, you might not get exactly the same
# output as below and that's fine. 
for i in [628, 766, 3338]:
    for xy in 'xy':
        print(data[i][xy][:2])
        print(docs_data[i][xy])
        print(SIMPLE_LESK_data[i][xy])
    print(f"{'':-^20}")



```
('sit', 'In West Bengal, the ruling Left Front organised sit-ins before the offices of oil PSUs.')
('NOUN', In West Bengal, the ruling Left Front organised sit-ins before the offices of oil PSUs.)
None
('stand', 'It was worth my while to try to stand up and punch with him.')
('VERB', It was worth my while to try to stand up and punch with him.)
Synset('digest.v.03')
--------------------
('family', 'Father Yus Mawengkang, a Catholic priest in Papua, said the family had been transported to the island on fishing boats and hoped that they would be collected by Australian authorities.')
('NOUN', Father Yus Mawengkang, a Catholic priest in Papua, said the family had been transported to the island on fishing boats and hoped that they would be collected by Australian authorities.)
Synset('syndicate.n.01')
('child', 'He is survived by three children, along with grandchildren and great-grandchildren.')
('NOUN', He is survived by three children, along with grandchildren and great-grandchildren.)
Synset('child.n.04')
--------------------
('goal', 'Scientific goals.')
('NOUN', Scientific goals.)
Synset('goal.n.04')
('soccer', "Fans of the Netherlands' soccer team were wildly celebrating too.")
('NOUN', Fans of the Netherlands' soccer team were wildly celebrating too.)
Synset('soccer.n.01')
--------------------
```



## Ex5[?pt]: Vector-based Lesk

The simple Lesk algorith uses the size of the word overlap between the context and the synset gloss to predict the synset. Here we ask you to use vector (cosine) similarity instead of the size of overlap. In sapCy is very simple to check vector similarity between two spaCy's tokens/spans/docs. But this also means that we need to process glosses of relevant synsets with spaCy.  
To make things relatively efficient, you can first extract all possibel glosses of target words of the data and process them with a singel `.pipe()` run of spaCy. It is up to you what will be gloss of teh synset, only examples, definitions, or both. Note that you can give several sentences as a singel string for processing to spaCy.  

After processing all relevant glosses, one needs to keep processed glosses related to their corresponding synsets. With the help of this, then you can easily retrieve processed glosses when checking similarity to the context of a target word. 

Read about Vector similarity in spaCy in [sec. 8](https://course.spacy.io/en/chapter2). More about the spaCy's vectors can be found [here](https://spacy.io/api/vectors).

In [ ]:
################################################################################
################################## EXERCISE 5 ##################################
################################################################################

def vector_lesk_sense(data, docs_data):
    """ Takes data and docs_data, and return a list of dictionaries
        {'x': synset_of_x, 'y': synset_of_y } where synset_of_x/y is 
        the sense of x/y in the context_x/y detecting by Lesk-like algorithm,
        which selects a sense based on the size of the overlap between the
        context and the definition and/or examples of the possible synsets.
        Note that pos tag of x/y is retrieved from docs_data and used
        to restric the possible senses for a word.
        The order in the returned list follows the orders in data and docs_data.
    """

    nlp = spacy.load("en_core_web_md")



In [ ]:
# if efficently implemented, takes <20sec
VEC_LESK_data = vector_lesk_sense(data, docs_data)

In [ ]:
#TEST EX5 
# These are the same data samples as the ones above.
# compare sense predictions from this to the previous function(s) and find the difference
# IMPORTANT: depending on certain choice points, you might not get exactly the same
# output as below and that's fine. 
for i in [628, 766, 3338]:
    for xy in 'xy':
        print(data[i][xy][:2])
        print(docs_data[i][xy])
        print(VEC_LESK_data[i][xy])
    print(f"{'':-^20}")

('sit', 'In West Bengal, the ruling Left Front organised sit-ins before the offices of oil PSUs.')
('NOUN', In West Bengal, the ruling Left Front organised sit-ins before the offices of oil PSUs.)
None
('stand', 'It was worth my while to try to stand up and punch with him.')
('VERB', It was worth my while to try to stand up and punch with him.)
Synset('digest.v.03')
--------------------
('family', 'Father Yus Mawengkang, a Catholic priest in Papua, said the family had been transported to the island on fishing boats and hoped that they would be collected by Australian authorities.')
('NOUN', Father Yus Mawengkang, a Catholic priest in Papua, said the family had been transported to the island on fishing boats and hoped that they would be collected by Australian authorities.)
Synset('family.n.08')
('child', 'He is survived by three children, along with grandchildren and great-grandchildren.')
('NOUN', He is survived by three children, along with grandchildren and great-grandchildren.)
Syn

## Ex6[?pt]: Mapping relations

To relate the word sense disambiguation to the task of predicting contextual lexical relation, we need to have a function that predicts one of the six lexical relations between two synsets. Based on the locations of the synsets in the WordNet, give a reasonable lexical relation between the synsets. For example, `forward_entailment`, `reverse_entailment`, and `equivalence` can be defined in terms of WordNet's hyponym/hypernym relations (see the note about the hypernymy/hyponymy relations below). `alternation` can be defined in terms of [co-hyponymy](https://en.wikipedia.org/wiki/Hyponymy_and_hypernymy#Co-hyponyms). `other-related` can be defined based on other WordNet relations. One can also try to use `similar_tos()` relation which stands for `similar to` for adjectives.

For using the hypernymy/hyponymy relation from WordNet, **you need too use its transitive version**.  

The hypernymy/hyponymy relation in WordNet doesn't come with transitivity closure. For example, `dog` has a sense `dog.n.01 (a member of the genus Canis ...)` and it is more specific than `animal.n.01 (a living organism ...)`, but this relation cannot be captured with `wn.synset('dog.n.01').hypernyms()` because it lists the immediate hypernyms: `canine.n.02'` and `domestic_animal.n.01`.  
The real picture in WordNet is the following: `dog.n.01` < `domestic_animal.n.01` < `animal.n.01` OR `dog.n.01` < `canine.02` < `carnivore.n.01` < ... < `chordate.n.01` < `animal.n.01`, where x < y denotes that x is more specific than y, i.e., x is a hyponym of y, i.e., y is a hypernym of x.  
We would like to capture the transitivity closure of the hypernymy relation. The NLTK's [howto](https://www.nltk.org/howto/wordnet.html) about WordNet will help you in this.  

Besides defining `lex_rel`, explain and motivate your WordNet-based definitions of lexical relations. Giving examples will help the explanation.

<font color="red">█████ MOTIVATION █████</font>

TYPE YOUR MOTIVATION HERE (don't delete the heaader)

In [ ]:
################################################################################
################################## EXERCISE 7 ##################################
################################################################################

def lex_rel(ss1, ss2):
    """ Takes two synsets and based on wordnet it outputs one of the following six
        relations: 'independent', 'equivalence', 'forward_entailment',
                   'reverse_entailment', 'alternation', 'other-related'
        if one of the sensets is None, it returns 'independent'.  
    """


    return 'independent' # fallback option

## Putting all together

Now we are putting together the various WSD functions and the definition of lexical relations between synsets based on WordNet.

In [ ]:
# in-context semantic relation classifier
def contextual_lex_rel(data, docs_data, WSD_OR_OUT):
    """ In addition to the raw data and processed data, it takes 
        the WSD function (most_frequent_sense, simple_lesk_sense, vector_lesk_sense)
        or its output and 
        Returns a list of predictions, i.e. one of teh six relations
    """ 
    # this construction is also called ternary operator
    # https://stackoverflow.com/questions/394809/does-python-have-a-ternary-conditional-operator
    wsd_data = WSD_OR_OUT if isinstance(WSD_OR_OUT, list) else WSD_OR_OUT(data, docs_data)
    # predciting relations for each data sample
    predicted_rels = [ lex_rel(d['x'], d['y']) for d in wsd_data ]
    return predicted_rels

In [ ]:
#TEST: the number of predictions should be the size of data
mfs_pred = contextual_lex_rel(data, docs_data, MFS_data)
print(f"Num of predictions = {len(mfs_pred)}")
# the predictions should only include at most six possible relations
print(f"A set of predictions = {set(mfs_pred)}")



```
Num of predictions = 3404
A set of predictions = {'equivalence', 'alternation', 'reverse_entailment', 'other-related', 'independent', 'forward_entailment'}
```



## Best performance

The code displays performance of all combinations of wsd components and the lexical relation definition. 

<font color="red">The groups of the top three perfroming systems will get additional bonus points (taking into account their positions `{1:5pt, 2:3pt, 3:2pt}`).</font>  
Note that most frequent sense (MFS) baseline is usually very good when it comes to word sense disambiguation. So, if your system is not better than the one based on MFS, it's ok. Remember that in general WSD is a hard task, especially when WSD is done wrt WordNet that has so many fine-grained synsets per word.

In [ ]:
#TEST final system evaluation
for wsd, name in zip([MFS_data, SIMPLE_LESK_data, VEC_LESK_data], 
               "mfs simple_lesk vector_lesk".split()):
    pred = contextual_lex_rel(data, docs_data, wsd)
    acc = evaluate_contextual_lex_rel(pred, data)
    print(f"{name}: {acc}")

# Reasoning Exercise TO BE ADDED

We show how reasoning is done with the natural tableau theorem prover, how certain inference relations are (in)correctly predicted, and how manually proving the lexical knowledge can help to find proofs.  
First we need to create LangPro object that knows locaton of LangPro's and SICK files. 